In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import json
import numpy as np
import pandas as pd

In [0]:
def preprocess(df):
  threshold=0.1
  df.replace(["", 'NaN'], np.nan, inplace = True)
  #remove columns with too many nan
  df=df[df.columns[df.isnull().mean()<threshold]]
  #fill nan with mode
  for col in df:
    df[col] = df[col].fillna(df[col].mode())
    '''
  #numerical outliers
  #not working
  for col in df.select_dtypes([np.number]):
    print('working')
    Q1 = col.quantile(0.02)
    Q3 = col.quantile(0.98)
    IQR = Q3 - Q1
    df= df[~((col< (Q1 - 1.5 * IQR)) |(col> (Q3 + 1.5 * IQR))).any(axis=1)]
    '''
  #all 0 columns
  df = df[(df.T != 0).any()]
  df=pd.DataFrame(df)
  return df

In [5]:
author=[]
with open ('/content/drive/My Drive/good_reads/goodreads_book_authors.json/goodreads_book_authors.json') as f:
  for line in f:
    author.append(json.loads(line))
author=pd.DataFrame(author)    
author.head()


,average_rating,author_id,text_reviews_count,name,ratings_count
0,3.98,604031,7,Ronald J. Fields,49
1,4.08,626222,28716,Anita Diamant,546796
2,3.92,10333,5075,Barbara Hambly,122118
3,3.68,9212,36262,Jennifer Weiner,888522
4,3.82,149918,96,Nigel Pennick,1740


In [0]:
author=preprocess(author)

In [0]:
author.to_csv('/content/drive/My Drive/good_reads/author.csv')

In [8]:
print(author.shape)
author.head()


(829529, 5)


,average_rating,author_id,text_reviews_count,name,ratings_count
0,3.98,604031,7,Ronald J. Fields,49
1,4.08,626222,28716,Anita Diamant,546796
2,3.92,10333,5075,Barbara Hambly,122118
3,3.68,9212,36262,Jennifer Weiner,888522
4,3.82,149918,96,Nigel Pennick,1740


In [9]:
author.describe()

,average_rating,author_id,text_reviews_count,name,ratings_count
count,829529,829529,829529,829524,829529
unique,377,829529,6241,825850,19686
top,4.00,933665,1,Phil Noto,1
freq,52411,1,158106,22,43799


In [6]:
genres=[]
count=0
with open ('/content/drive/My Drive/good_reads/goodreads_book_genres_initial.json/goodreads_book_genres_initial.json') as f:
  for line in f:
    genres.append(json.loads(line))
    count+=1
    if count>1000000:
      break
genres=pd.DataFrame(genres)    
genres.head()

,book_id,genres
0,5333265,"{'history, historical fiction, biography': 1}"
1,1333909,"{'fiction': 219, 'history, historical fiction,..."
2,7327624,"{'fantasy, paranormal': 31, 'fiction': 8, 'mys..."
3,6066819,"{'fiction': 555, 'romance': 23, 'mystery, thri..."
4,287140,{'non-fiction': 3}


In [7]:
genres.shape

(1000001, 2)

In [0]:
genres=preprocess(genres) #too large, ram not enough
genres.to_csv('/content/drive/My Drive/good_reads/genres.csv')

In [0]:
book_genre=[]
for bid, gen in zip(genres['book_id'],genres['genres']):
    #print(gen)
    if gen !='{}' and gen!= {}:
      book_genre.append([bid,max(gen, key=gen.get)])


In [14]:
book_genre=pd.DataFrame(book_genre)
book_genre.head()
book_genre.shape

(826331, 2)

In [0]:
book_genre.to_csv('/content/drive/My Drive/good_reads/book_genre.csv')

In [19]:
series=[]
with open ('/content/drive/My Drive/good_reads/goodreads_book_series.json/goodreads_book_series.json') as f:
  for line in f:
    series.append(json.loads(line))
series=pd.DataFrame(series)    
series.head()

,numbered,note,description,title,series_works_count,series_id,primary_work_count
0,true,,,Sun Wolf and Starhawk,9,189911,3
1,true,,This series is also known as * Avalon : Jalina...,Avalon: Web of Magic,14,151854,12
2,true,,"Plot-wise, ""Crowner's Crusade"" is a prequel to...",Crowner John Mystery,15,169353,15
3,true,,,Pluto's Snitch,3,1052227,3
4,false,,,"The Way, With The Enchanted Neighborhood",3,1070125,3


In [0]:
series=preprocess(series)

In [0]:
series.to_csv('/content/drive/My Drive/good_reads/series.csv')

In [23]:
series.isnull().sum(axis=0)
series.describe()


,numbered,title,series_works_count,series_id,primary_work_count
count,400390,400384,400390,400390,400390
unique,2,101433,250,400390,241
top,true,Don't Read in the Closet Events,3,905475,3
freq,356008,711,58330,1,74342


In [24]:
series.shape

(400390, 5)

In [25]:
works=[]
with open ('/content/drive/My Drive/good_reads/goodreads_book_works.json/goodreads_book_works.json') as f:
  for line in f:
    works.append(json.loads(line))
works=pd.DataFrame(works)    
works.head()

,books_count,reviews_count,original_publication_month,default_description_language_code,text_reviews_count,best_book_id,original_publication_year,original_title,rating_dist,default_chaptering_book_id,original_publication_day,original_language_id,ratings_count,media_type,ratings_sum,work_id
0,1,6,8,,1,5333265,1984,W. C. Fields: A Life on Film,5:1|4:1|3:1|2:0|1:0|total:3,,,,3,book,12,5400751
1,22,10162,,,741,25717,2001,Good Harbor,5:517|4:1787|3:2763|2:966|1:196|total:6229,,,,6229,book,20150,1323437
2,2,268,,,7,7327624,1987,,5:49|4:58|3:26|2:5|1:3|total:141,,,,141,book,568,8948723
3,38,89252,7,,3504,6066819,2009,Best Friends Forever,5:9152|4:16855|3:19507|2:6210|1:1549|total:53273,,14,,53273,book,185670,6243154
4,2,49,,,5,287140,1990,Runic Astrology: Starcraft and Timekeeping in ...,5:6|4:1|3:3|2:3|1:2|total:15,,,,15,book,51,278577


In [26]:
works=preprocess(works)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [0]:
works.to_csv('/content/drive/My Drive/good_reads/works.csv')

In [28]:
works.head()

,books_count,reviews_count,text_reviews_count,best_book_id,original_publication_year,rating_dist,ratings_count,media_type,ratings_sum,work_id
0,1,6,1,5333265,1984,5:1|4:1|3:1|2:0|1:0|total:3,3,book,12,5400751
1,22,10162,741,25717,2001,5:517|4:1787|3:2763|2:966|1:196|total:6229,6229,book,20150,1323437
2,2,268,7,7327624,1987,5:49|4:58|3:26|2:5|1:3|total:141,141,book,568,8948723
3,38,89252,3504,6066819,2009,5:9152|4:16855|3:19507|2:6210|1:1549|total:53273,53273,book,185670,6243154
4,2,49,5,287140,1990,5:6|4:1|3:3|2:3|1:2|total:15,15,book,51,278577


In [29]:
works.shape

(1521962, 10)

In [30]:
works.describe()

,books_count,reviews_count,text_reviews_count,best_book_id,original_publication_year,rating_dist,ratings_count,media_type,ratings_sum,work_id
count,1521962,1521962,1521962,1521962,1269387,1521962,1521962,1107866,1521962,1521962
unique,628,27553,4794,1521962,1263,691714,19463,4,37952,1521962
top,1,1,1,15747359,2014,5:1|4:0|3:0|2:0|1:0|total:1,1,book,5,47430455
freq,518515,39549,333552,1,125241,31763,94008,1102395,36748,1


In [9]:
reviews=[]
count=0
with open ('/content/drive/My Drive/good_reads/goodreads_reviews_dedup.json/goodreads_reviews_dedup.json') as f:
  for line in f:
    reviews.append(json.loads(line))
    count+=1
    if count>1000000:
      break
reviews=pd.DataFrame(reviews)    
reviews.head()

,user_id,book_id,review_id,rating,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments
0,8842281e1d1347389f2ab93d60773d4d,24375664,5cd416f3efc3f944fce4ce2db2290d5e,5,Mind blowingly cool. Best science fiction I've...,Fri Aug 25 13:55:02 -0700 2017,Mon Oct 09 08:55:59 -0700 2017,Sat Oct 07 00:00:00 -0700 2017,Sat Aug 26 00:00:00 -0700 2017,16,0
1,8842281e1d1347389f2ab93d60773d4d,18245960,dfdbb7b0eb5a7e4c26d59a937e2e5feb,5,This is a special book. It started slow for ab...,Sun Jul 30 07:44:10 -0700 2017,Wed Aug 30 00:00:26 -0700 2017,Sat Aug 26 12:05:52 -0700 2017,Tue Aug 15 13:23:18 -0700 2017,28,1
2,8842281e1d1347389f2ab93d60773d4d,6392944,5e212a62bced17b4dbe41150e5bb9037,3,I haven't read a fun mystery book in a while a...,Mon Jul 24 02:48:17 -0700 2017,Sun Jul 30 09:28:03 -0700 2017,Tue Jul 25 00:00:00 -0700 2017,Mon Jul 24 00:00:00 -0700 2017,6,0
3,8842281e1d1347389f2ab93d60773d4d,22078596,fdd13cad0695656be99828cd75d6eb73,4,"Fun, fast paced, and disturbing tale of murder...",Mon Jul 24 02:33:09 -0700 2017,Sun Jul 30 10:23:54 -0700 2017,Sun Jul 30 15:42:05 -0700 2017,Tue Jul 25 00:00:00 -0700 2017,22,4
4,8842281e1d1347389f2ab93d60773d4d,6644782,bd0df91c9d918c0e433b9ab3a9a5c451,4,A fun book that gives you a sense of living in...,Mon Jul 24 02:28:14 -0700 2017,Thu Aug 24 00:07:20 -0700 2017,Sat Aug 05 00:00:00 -0700 2017,Sun Jul 30 00:00:00 -0700 2017,8,0


In [10]:
reviews=preprocess(reviews)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [0]:
reviews.to_csv('/content/drive/My Drive/good_reads/review.csv')